In [ ]:
'''
author: Felix Hol
date: 2019 Aug 20
content: code to track mosquitoes, create crops, creat presence / stylet ethogram
'''

import numpy as np
%matplotlib inline
import matplotlib as mpl
# mpl.use('TkAgg') ### this is a workaround for a bug when running on macOS, may not be necessary anymore
import matplotlib.pyplot as plt
import os
import itertools as it
import pandas as pd
from pandas import DataFrame, Series  # for convenience
import pims
import skimage
from skimage import data, io, util
from skimage.feature import (canny, match_descriptors, peak_local_max, match_template, corner_peaks, corner_harris, plot_matches, BRIEF)
from skimage.color import rgb2gray
from skimage.draw import circle
from skimage.measure import label, regionprops
from skimage.morphology import binary_dilation, erosion, dilation, opening, binary_closing, closing, white_tophat, remove_small_objects, disk, black_tophat, skeletonize, convex_hull_image
from scipy import ndimage as ndi
import scipy
import trackpy as tp
import pylab
import math
from joblib import Parallel, delayed
import multiprocessing
from datetime import datetime
from tqdm import tnrange
import pickle
import glob

In [ ]:
dataDir = '/Volumes/Fe_ssd02/bloodTaste/190702_SalineNoATP/M7/'
saveDir = '/Users/felix/Documents/mosquitoes/mosquitoData/bloodTasteData/analysis/SalineNoATP/190702_SalineNoATP_M7/'
frames = pims.ImageSequence(dataDir+'/*.tiff', as_grey=True)

In [ ]:
len(frames)

In [ ]:
####create background image

start = 1423
stop = len(frames) - 1
step = math.floor(len(frames)/20)

numBGframes = int(np.ceil((stop - start) /step) + 1)
frameSize = frames[1].shape
BG = np.zeros([frameSize[0], frameSize[1], numBGframes])

j = 1
for i in range(start, stop, step):
    j += 1
    BG[:, :, j - 1] = np.invert(frames[i])    

BG = np.median(BG, axis=2)

In [ ]:
plt.figure(figsize=(25,10))
plt.imshow(BG)

In [ ]:
#### get centroid coordinates

def trackMosq2(i):
    borderToExclude = 50
    selem1 = disk(8)
    selem2 = disk(1)
    A = np.zeros(frameSize)
    A = A + np.invert(frames[i])
    B = A - BG
    if B.min() > 0:
        Bm = B - B.min()
    else:
        Bm = B
    Bt = Bm > 90
    Bts = remove_small_objects(Bt, min_size=400)
    Be = erosion(Bts, selem2)
    Bf = remove_small_objects(Be, min_size=300)
    Bc = binary_closing(Bf, selem1)
    C = B * Bc
    eroded = erosion(C, selem2)
    eroded = skimage.filters.gaussian(eroded, 4)
    eroded[eroded < 0] = 0
    erL = label(eroded>0)
    erR = regionprops(erL, C)#, coordinates='xy')
    l = 1
    for props in erR:
        if props.area > 100000:
            erL[erL==l] = 0
        if props.area < 1000:
            erL[erL==l] = 0
        if props.major_axis_length > 1000:
            erL[erL==l] = 0
        if props.centroid[0] < borderToExclude or props.centroid[1] < borderToExclude or props.centroid[0] > frameSize[0] - borderToExclude or props.centroid[1] > frameSize[1] - borderToExclude:
            erL[erL==l] = 0
        l = l +1
    erLf = label(erL>0)
    erodedF = eroded * (erLf > 0)
    erRf = regionprops(erLf, C)#, coordinates='xy')
    centroids = np.zeros([len(erRf), 2])
    numCent = 0
    for props in erRf:
        centroids[numCent] = props.centroid
        numCent += 1
#     coordinates = peak_local_max(eroded, min_distance=130, exclude_border=1)
#     cS= coordinates.shape; numCoor = cS[0]
    cenS= centroids.shape; numCen = cenS[0]
    frameNo = i
#     frameNoA = np.zeros((numCoor,1), dtype=np.int)
    frameNoCen = np.zeros((numCen,1), dtype=np.int)
#     frameNoA[:] = frameNo
    frameNoCen[:] = frameNo
#     coordinatesF = np.hstack((coordinates,frameNoA))
    centroidsF = np.hstack((centroids,frameNoCen))
    if len(centroids) > 1:
        diffX = np.abs(centroids[0][0] - centroids[1][0])
        diffY = np.abs(centroids[0][1] - centroids[1][1])
        if diffX < 250 and diffY < 250:
            AA = np.zeros([1,3])
            AA[0][0] = np.mean([centroidsF[0][0], centroidsF[1][0]]) 
            AA[0][1] = np.mean([centroidsF[0][1], centroidsF[1][1]])
            AA[0][2] = np.mean(frameNoCen)
            centroidsF = AA
#     coordinatesF = np.hstack((coordinates,frameNoA))
#     numCoords = coordinatesF.shape[0]
    numCents = centroidsF.shape[0]
    return centroidsF, numCents

#     return centroidsF, coordinatesF, numCents, numCoords


In [ ]:
#####testing....
i = 1788
centroidsF, numCents = trackMosq2(i)
plt.figure(figsize=(18,12))
plt.imshow(frames[i], cmap = 'gray')
# plt.imshow(eroded)
# plt.plot(coordinatesF[:,1],coordinatesF[:,0],'r.')
plt.plot(centroidsF[:,1],centroidsF[:,0],'r.')
centroidsF

In [ ]:
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(trackMosq2)(i) for i in tnrange(1426, len(frames)))

In [ ]:
centroidsAllT = np.zeros((1,3))

for i in range(len(results)):
    centroidsAllT = np.vstack((centroidsAllT,results[i][0]))

In [ ]:
len(centroidsAllT)

In [ ]:
plt.figure(figsize=(18,12))
plt.imshow(frames[1000], cmap = 'gray')
# plt.imshow(eroded)
# plt.plot(coordinatesF[:,1],coordinatesF[:,0],'r.')
plt.plot(centroidsAllT[:,1],centroidsAllT[:,0],'r.')

In [ ]:
os.chdir(saveDir)
with open('190702_SalineNoATP_M7_ROICent.pkl', 'wb') as f:
    pickle.dump(centroidsAllT, f)

In [ ]:
# os.chdir(saveDir)
# with open('190625_KPPTN_01_Cent.pkl', 'rb') as f:
#     centroidsAllT = pickle.load(f)

In [ ]:
df_cenAllT = pd.DataFrame(centroidsAllT)
df_cenAllT = df_cenAllT.rename(columns={0: "y", 1: "x", 2: "frame"})
df_cenAllT.head()

In [ ]:
t = tp.link_df(df_cenAllT, 300, memory=2)

In [ ]:
plt.figure(figsize=(25,10))
plt.imshow(frames[200], cmap='gray')
plt.plot(centroidsAllT[:,1],centroidsAllT[:,0],'r.',alpha=0.5, markersize=10)
tp.plot_traj(t, plot_style={'linewidth':'2.0'})

In [ ]:
tFilt = tp.filter_stubs(t, 10)

In [ ]:
fig = plt.figure(figsize=(30,10))
ax = fig.add_subplot(1, 1, 1)
# fig.figure(figsize=(10,50))

for i in tFilt.particle.unique():
    start = np.min(tFilt.loc[tFilt['particle'] == i].frame)
    width = np.max(tFilt.loc[tFilt['particle'] == i].frame) - np.min(tFilt.loc[tFilt['particle'] == i].frame)
    rect = plt.Rectangle((start, 0.75), width, 0.15, color='r', alpha=0.2, ec=None)
    ax.add_patch(rect)

plt.xlim([0, 9000])

In [ ]:
os.chdir(saveDir)
t.to_pickle('190702_SalineNoATP_M7_ROItracks.pkl')

In [ ]:
# os.chdir(saveDir)
# t = pickle.load(open('190625_KPPTN_01_tracks.pkl', 'rb'))

In [ ]:
t.head()

In [ ]:
tFilt = tp.filter_stubs(t, 4)
tFilt.particle.unique()

In [ ]:
len(tFilt)

In [ ]:
def cropMos(p, t):
    
    frameWidth = 2048
    frameHeigth = 2048
    ROIwidth = 550
    ROIheigth = 550
    halfROIwidth = ROIwidth / 2
    halfROIheigth = ROIheigth / 2
    
    tP = t.loc[t['particle'] == p]
    for index, row in tP.iterrows():
        if row['x'] < halfROIwidth:
            x_start = 0
            x_stop = ROIwidth
        elif row['x'] + halfROIwidth > frameWidth:
            x_start = frameWidth - ROIwidth
            x_stop = frameWidth
        else:
            x_start = row['x'] - halfROIwidth
            x_stop = row['x'] + halfROIwidth
        if row['y'] < halfROIwidth:
            y_start = 0
            y_stop = ROIwidth
        elif row['y'] + halfROIwidth > frameWidth:
            y_start = frameWidth - ROIwidth
            y_stop = frameWidth
        else:
            y_start = row['y'] - halfROIwidth
            y_stop = row['y'] + halfROIwidth
        currFrame = frames[row['frame']]
        currROI = currFrame[int(y_start):int(y_stop), int(x_start):int(x_stop)]
        skimage.io.imsave("crop_p" + str(int(p)).zfill(3) + "_f" + str(int(row['frame'])).zfill(6) + ".png", currROI)


In [ ]:
os.chdir(saveDir + 'crops/')
mosToCrop = tFilt.particle.unique()

num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(cropMos)(i, t) for i in mosToCrop)

In [ ]:
startPierce = [1924, 4064, 4376, 4486, 4601, 4648]
stopPierce = [1949, 4297, 4436, 4512, 4628, 4673]

In [ ]:
len(startPierce)

In [ ]:
tp = t.copy()
tp['stylet'] = 0

for i in range(0,len(startPierce) - 1):
    start = startPierce[i]
    stop = stopPierce[i]
    tp.loc[(tp.frame >= start) & (tp.frame <= stop), 'stylet'] = 1

In [ ]:
os.chdir(saveDir)
tp.to_pickle('190702_SalineNoATP_M7_ROItracksStylet.pkl')

In [ ]:
tsd = tp.copy()

fig = plt.figure(figsize=(22,3))
ax = fig.add_subplot(1, 1, 1)

for i in tsd.particle.unique():
    start = np.min(tsd.loc[ts['particle'] == i].frame)
    width = np.max(tsd.loc[ts['particle'] == i].frame) - np.min(tsd.loc[ts['particle'] == i].frame)
    rect = plt.Rectangle((start, 1), width, 0.1, color='c', alpha=0.65, ec=None)
    ax.add_patch(rect)

p = tsd.loc[tsd.stylet == 1].frame.values
ph = np.full((len(p)), 0.95)
P = [p,ph]
P_zipped = zip(*P)
for P_x, P_y in P_zipped:
    ax.add_patch(plt.Rectangle((P_x, P_y), 1, 0.1, color='m', alpha=0.75, ec=None))

ax.set_xticklabels(list(range(0,1000, 100)))
ax.set_yticklabels([])
plt.xlabel('time (s)')

cyan_patch = mpl.patches.Patch(color='c', label='mosquito present')
magenta_patch = mpl.patches.Patch(color='m', label='stylet piercing')
plt.legend(handles=[cyan_patch, magenta_patch])
    
plt.xlim([0, 9000])
plt.ylim([0.8, 1.2])

In [ ]:
plt.figure(figsize=(25,10))
plt.imshow(frames[200], cmap='gray')
plt.plot(centroidsAllT[:,1],centroidsAllT[:,0],'r.',alpha=0.5, markersize=10)
tp.plot_traj(tFilt, plot_style={'linewidth':'2.0'})

In [ ]:
len(tFilt.particle.unique())

In [ ]:
plt.figure(figsize=(25,10))
plt.imshow(frames[50], cmap='gray')
tp.plot_traj(tFilt.loc[tFilt['particle'] == 17.0], plot_style={'linewidth':'3.0'})

In [ ]:
os.chdir(saveDir)
pickleList = glob.glob('*.pkl')
centroidsAllCat = np.zeros((1,3))

for i in pickleList:
    with open(i, "rb") as input_file:
        centroidsI = pickle.load(input_file)
    centroidsAllCat = np.vstack((centroidsAllCat, centroidsI))

In [ ]:
np.max(B)

In [ ]:

A = np.zeros(frameSize)
A = A + np.invert(frames[4381])
B = A - BG
B[B < 0] = 0
# if B.min() > 0:
#     Bm = B - B.min()
# else:
#     Bm = B

plt.figure(figsize=(25,25))
plt.imshow(np.invert(B.astype(int)), cmap='gray', vmin=25, vmax=95)

In [ ]:
plt.figure(figsize=(25,25))

alphas = [1, 0.5, 0.35, 0.25]

j = 0

for i in [3194, 3312, 4675, 5767]:
    A = np.zeros(frameSize)
    A = A + np.invert(frames[i])
    B = A - BG
    B[B < 0] = 0
    plt.imshow(np.invert(B.astype(int)), cmap='gray', alpha=alphas[j], vmin=-40, vmax=-3)
    j = j + 1
    
plt.savefig('190625_KPTTN_p049_114_125_NOtracks05.png')

In [ ]:
plt.figure(figsize=(25,25))

alphas = [1, 0.75, 0.65, 0.55]

j = 0

for i in [3194, 3312, 4675, 5767]:
    A = np.zeros(frameSize)
    A = A + np.invert(frames[i])
    B = A - BG
    B[B < 0] = 0
    plt.imshow(np.invert(B.astype(int)), cmap='gray', alpha=alphas[j], vmin=-40, vmax=-3)
    j = j + 1

for i in range(1, len(p114) - 2):
    d = math.sqrt((p114.iloc[i + 1].x - p114.iloc[i].x) ** 2 + (p114.iloc[i + 1].y - p114.iloc[i].y) ** 2)
    plt.plot(p114.iloc[i:i+2].x, p114.iloc[i:i+2].y, c=mpl.cm.autumn(1 / (d/15)), lw=5)
    
for i in range(1, len(p049) - 2):
    d = math.sqrt((p049.iloc[i + 1].x - p049.iloc[i].x) ** 2 + (p049.iloc[i + 1].y - p049.iloc[i].y) ** 2)
    plt.plot(p049.iloc[i:i+2].x, p049.iloc[i:i+2].y, c=mpl.cm.autumn(1 / (d/15)), lw=5)
    
for i in range(1, len(p125) - 2):
    d = math.sqrt((p125.iloc[i + 1].x - p125.iloc[i].x) ** 2 + (p125.iloc[i + 1].y - p125.iloc[i].y) ** 2)
    plt.plot(p125.iloc[i:i+2].x, p125.iloc[i:i+2].y, c=mpl.cm.autumn(1 / (d/15)), lw=5)
    
plt.savefig('190625_KPTTN_p049_114_125_tracks05.pdf')

In [ ]:
os.getcwd()

In [ ]:
plt.figure(figsize=(25,25))
plt.imshow(frames[3194], cmap='gray', vmin=125, vmax=155, alpha=1)
plt.imshow(frames[3312], cmap='gray', vmin=125, vmax=155, alpha=0.5)
plt.imshow(frames[5100], cmap='gray', vmin=125, vmax=155, alpha=0.4)
# plt.imshow(frames[2591], cmap='gray', vmin=55, vmax=160, alpha=0.4)
# plt.imshow(frames[5571], cmap='gray', vmin=55, vmax=160, alpha=0.4)
# plt.imshow(frames[3195], cmap='gray', vmin=55, vmax=180, alpha=0.4)

# plt.imshow(frames[4327], cmap='gray', vmin=95, vmax=180, alpha=1)
# plt.imshow(frames[4411], cmap='gray', vmin=95, vmax=180, alpha=0.5)
# plt.imshow(frames[5148], cmap='gray', vmin=55, vmax=180, alpha=0.4)

# for i in range(1, len(p114) - 2):
#     d = math.sqrt((p114.iloc[i + 1].x - p114.iloc[i].x) ** 2 + (p114.iloc[i + 1].y - p114.iloc[i].y) ** 2)
#     plt.plot(p114.iloc[i:i+2].x, p114.iloc[i:i+2].y, c=mpl.cm.autumn(1 / (d/15)), lw=5)
    
# for i in range(1, len(p049) - 2):
#     d = math.sqrt((p049.iloc[i + 1].x - p049.iloc[i].x) ** 2 + (p049.iloc[i + 1].y - p049.iloc[i].y) ** 2)
#     plt.plot(p049.iloc[i:i+2].x, p049.iloc[i:i+2].y, c=mpl.cm.autumn(1 / (d/15)), lw=5)
    
# for i in range(1, len(p125) - 2):
#     d = math.sqrt((p125.iloc[i + 1].x - p125.iloc[i].x) ** 2 + (p125.iloc[i + 1].y - p125.iloc[i].y) ** 2)
#     plt.plot(p125.iloc[i:i+2].x, p125.iloc[i:i+2].y, c=mpl.cm.autumn(1 / (d/15)), lw=5)